In [1]:
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, MinMaxScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
import xgboost as xgb
import joblib

# Paths to datasets
train_path = r"C:\Users\paulo\OneDrive\Documents\kaggle_competition_2_datasets\train.csv"
test_path = r"C:\Users\paulo\OneDrive\Documents\kaggle_competition_2_datasets\test.csv"

def import_data(path, index_col=None):
    """Import data from a CSV file and optimize memory usage."""
    df = pd.read_csv(path, index_col=index_col)
    return reduce_mem_usage(df)

def reduce_mem_usage(df):
    """Iterate through all the columns of a dataframe and modify the data type to reduce memory usage."""
    for col in df.columns:
        col_type = df[col].dtype
        if isinstance(col_type, pd.IntervalDtype):
            continue

        if str(col_type)[:3] == 'int':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                df[col] = df[col].astype(np.int64)  
        elif str(col_type)[:5] == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
    return df

def feature_engineering(df):
    """Feature engineering on the dataset."""
    # Binning age and converting to categorical labels instead of intervals
    age_bins = pd.cut(df['Age'], bins=7, labels=False)
    df['Age_Type'] = age_bins
    df['Vehicle_Age'] = df['Vehicle_Age'].astype('category').cat.codes
    df['Vehicle_Damage'] = df['Vehicle_Damage'].astype('category').cat.codes
    df['Previously_Insured'] = df['Previously_Insured'].astype('category').cat.codes

    df['Age_x_Vehicle_Age'] = df['Age_Type'] * df['Vehicle_Age']
    df['Age_x_Vehicle_Damage'] = df['Age_Type'] * df['Vehicle_Damage']
    df['Age_x_Previously_Insured'] = df['Age_Type'] * df['Previously_Insured']

    fac_pre = ['Policy_Sales_Channel', 'Vehicle_Damage', 'Annual_Premium', 'Vintage', 'Age_Type']
    col_pre = []
    for i in fac_pre:
        df['Previously_Insured_x_' + i] = pd.factorize(df['Previously_Insured'].astype(str) + df[i].astype(str))[0]
        col_pre.append('Previously_Insured_x_' + i)

    fac_pro = fac_pre[1:]
    col_pro = []
    for i in fac_pro:
        df['Policy_Sales_Channel_x_' + i] = pd.factorize(df['Policy_Sales_Channel'].astype(str) + df[i].astype(str))[0]
        col_pro.append('Policy_Sales_Channel_x_' + i)
    return df, col_pre, col_pro


In [2]:
# Load and optimize data
train_df = import_data(train_path, index_col='id')
test_df = import_data(test_path, index_col='id')

# Combine train and test datasets for consistent transformation
full_df = pd.concat([train_df, test_df], axis=0)

# Convert columns to category type
less = ['Gender', 'Vehicle_Age', 'Vehicle_Damage', 'Policy_Sales_Channel']
for col in less:
    full_df[col] = full_df[col].astype('category')

# Apply feature engineering to the combined dataset
full_df, col_pre, col_pro = feature_engineering(full_df)


In [3]:
# Split back into train and test sets
train_df = full_df.iloc[:len(train_df), :]
test_df = full_df.iloc[len(train_df):, :]

# Split the training data into training and validation sets
X = train_df.drop('Response', axis=1)
y = train_df['Response']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [4]:
# Define the ColumnTransformer
coltrans = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=False, dtype=np.float32), ['Gender', 'Vehicle_Damage']),
        ('minmax', MinMaxScaler(), ['Age', 'Region_Code', 'Previously_Insured', 'Policy_Sales_Channel', 'Vintage']),
        ('ordinal', OrdinalEncoder(categories=[[0, 1, 2]], dtype=np.float32), ['Vehicle_Age']),
        ('robust', RobustScaler(), ['Annual_Premium']),
        ('standard', StandardScaler(), ['Age_Type', 'Age_x_Vehicle_Age', 'Age_x_Vehicle_Damage', 'Age_x_Previously_Insured']),
        ('standard_2', StandardScaler(), col_pre + col_pro),
    ],
    remainder='passthrough'  # Keeps columns not specified in transformers
)

# Fit the transformer on the training data and transform both training and validation sets
X_train = coltrans.fit_transform(X_train)
X_valid = coltrans.transform(X_valid)
test_df = coltrans.transform(test_df.drop('Response', axis=1))

gc.collect()

123

In [5]:
# Calculate the ratio for scale_pos_weight
ratio = len(train_df[train_df['Response'] == 0]) / len(train_df[train_df['Response'] == 1])

# Define XGBoost parameters
xgb_params = {
    'random_state': 512,
    'objective': "binary:logistic",
    'eval_metric': 'auc',
    'learning_rate': 0.12,
    'max_bin': 225000,
    'subsample': 0.8,
    'reg_alpha': 0.15,
    'min_child_weight': 14,
    'colsample_bytree': 0.6,
    'gamma': 0.1,
    'reg_lambda': 0.75,
    'max_depth': 8,
    'scale_pos_weight': ratio,
    # 'tree_method': 'hist',
    # 'device': 'cuda',
}

# Initialize lists to store out-of-fold predictions and AUC scores
xgb_preds = []
xgb_aucs = []

# Train XGBoost model with cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, valid_idx) in enumerate(skf.split(X_train, y_train)):
    print(f"Training fold {fold + 1}")
    X_train_fold, y_train_fold = X_train[train_idx], y_train.iloc[train_idx]
    X_valid_fold, y_valid_fold = X_train[valid_idx], y_train.iloc[valid_idx]
    
    dtrain = xgb.DMatrix(X_train_fold, label=y_train_fold)
    dvalid = xgb.DMatrix(X_valid_fold, label=y_valid_fold)
    
    model = xgb.train(
        xgb_params,
        dtrain,
        num_boost_round=3000,
        evals=[(dtrain, 'train'), (dvalid, 'valid')],
        verbose_eval=100,
        early_stopping_rounds=10,
    )
    
    valid_preds = model.predict(dvalid, iteration_range=(0, model.best_iteration))
    auc_score = roc_auc_score(y_valid_fold, valid_preds)
    xgb_aucs.append(auc_score)
    
    dtest = xgb.DMatrix(test_df)
    test_pred = model.predict(dtest, iteration_range=(0, model.best_iteration))
    xgb_preds.append(test_pred)
    
    # Save the model for this fold
    model.save_model(f'xgb_model_fold_{fold + 1}.json')
    
    # Clear memory
    del X_train_fold, y_train_fold, X_valid_fold, y_valid_fold, dtrain, dvalid, model
    gc.collect()

# Calculate overall AUC score for XGBoost
auc_mean_xgb = np.mean(xgb_aucs)
auc_std_xgb = np.std(xgb_aucs)

Training fold 1
[0]	train-auc:0.85397	valid-auc:0.85421


KeyboardInterrupt: 

In [ ]:
# Average the predictions from each fold for XGBoost
test_pred_xgb = np.mean(xgb_preds, axis=0)
joblib.dump(test_pred_xgb, 'test_pred_xgb3.pkl')


['test_pred_xgb.pkl']

In [ ]:
# Reimport the test_df to get the original index
test_df = import_data(test_path, index_col='id')

# Create a submission DataFrame using the original test index
submission = pd.DataFrame({
    'id': test_df.index,
    'Response': test_pred_xgb
})

# Save the submission DataFrame to a CSV file
submission.to_csv('submission_xgb3.csv', index=False)


C:\Users\paulo\AppData\Local\Temp\ipykernel_10884\3486884307.py:26: DeprecationWarning: is_interval_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.IntervalDtype)` instead
  if pd.api.types.is_interval_dtype(col_type):
C:\Users\paulo\AppData\Local\Temp\ipykernel_10884\3486884307.py:26: DeprecationWarning: is_interval_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.IntervalDtype)` instead
  if pd.api.types.is_interval_dtype(col_type):
C:\Users\paulo\AppData\Local\Temp\ipykernel_10884\3486884307.py:26: DeprecationWarning: is_interval_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.IntervalDtype)` instead
  if pd.api.types.is_interval_dtype(col_type):
C:\Users\paulo\AppData\Local\Temp\ipykernel_10884\3486884307.py:26: DeprecationWarning: is_interval_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.IntervalDtype)` instead
  if pd.api.typ